In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky 
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from scipy import constants
from scipy import stats as sci_stats  
import nawrapper as nw
import math
import csv 

path = "/home/kw4043/project/Planck_cib_maps/"
path_beam = "/home/kw4043/project/Planck_cib_maps/beams/BeamWf_HFI_R3/"

path_HI_857_40 = "/home/kw4043/project/Planck_cib_maps/HI_maps/4.0e+20_gp40_857/"

##I'm gonna be using path_beam for the mcm's as well to separate from the previous notebooks

def eshow(x,**kwargs): 
    ''' Define a function to help us plot the maps neatly '''
    plots = enplot.get_plots(x, **kwargs)
    enplot.show(plots, method = "ipython")
    
def shrink_mask(mask, r):
    """Shrink the True part of boolean mask "mask" by a distance of r radians"""
    return mask.distance_transform(rmax=r) >= r

def coth(x):
    "function returns value for hyperbolic cotangent"
    return 1/np.tanh(x)

In [ ]:
#Creating the bins then converting our maps into namap_car objects
bins = nw.create_binning(lmax=3000, lmin=2,
                         widths=[15]*2 + [20]*3 + [25]*4 + [75]*2 + 1*[100] + 2*[150] + 11*[200],
                         weight_function=(lambda ell : ell**2))
#Our box for maps
box = [[np.pi/2, -np.pi/2],[-np.pi, np.pi]]
imap = curvedsky.make_projectable_map_by_pos(box, lmax=3000, dims=(), oversample=2.0, dtype=float, verbose=False)

bl_y = hp.gauss_beam(np.radians(10/60), lmax = 3000)

### Loading in the beam files for HI maps

In [ ]:
from astropy.io import ascii
beam_HI_857_fn_40 = path_HI_857_40 + 'windowfunctions.csv'
beam_HI_857_fn_40 = ascii.read(beam_HI_857_fn_40)
beam_HI_857_40 = beam_HI_857_fn_40['Wl_eff']
beam_HI_857_40 = np.array(beam_HI_857_40)
print(beam_HI_857_40)

### Using the deproj cib and the non-deproj Y maps

In [ ]:
bl_ACT = hp.gauss_beam(np.radians(1.5/60), lmax = 3000)

In [ ]:
ymap_fname_1 = path + "ilc_SZ_TT_masked.fits"
ACT_Y_map_1 = enmap.read_map(ymap_fname_1)
ACT_Y_map_1 = enmap.downgrade(ACT_Y_map_1, 4)

In [ ]:
ymap_fname_2 = path + "ilc_SZ_deproj_cib_TT_masked.fits"
ACT_Y_map_2_cib = enmap.read_map(ymap_fname_2)
ACT_Y_map_2_cib = enmap.downgrade(ACT_Y_map_2_cib, 4)

### Reading in HI maps 

In [ ]:
pmap_fname_HI_857_40 = path_HI_857_40 + "cib_fullmission.hpx.fits"
HI_857_map_40 = hp.read_map(pmap_fname_HI_857_40, field=0)

#reading in the mask they used
pmap_fname_HI_857_mask_40 = path_HI_857_40 + "mask_bool.hpx.fits"
HI_857_mask_40 = hp.read_map(pmap_fname_HI_857_mask_40, field=0)

In [ ]:
#seeing if applying the mask works
HI_857_map_with_mask_40 = HI_857_mask_40.copy()
trial = np.where(HI_857_map_with_mask_40 == 1)
HI_857_map_with_mask_40[trial] = HI_857_map_40[trial]

In [ ]:
HI_857_map_enmap_40 = reproject.enmap_from_healpix(HI_857_map_with_mask_40, ACT_Y_map_1.shape, ACT_Y_map_1.wcs,
                                  ncomp=1, unit=1, lmax=3000,rot='gal,equ')

HI_857_mask_enmap_40 = reproject.enmap_from_healpix(HI_857_mask_40, ACT_Y_map_1.shape, ACT_Y_map_1.wcs,
                                  ncomp=1, unit=1, lmax=3000,rot='gal,equ')

### Unit Conversions

In [ ]:
#Converting frequency maps from MJy/sr to Kelvin
HI_857_map_enmap_40 = HI_857_map_enmap_40 / 2.27 

#Converting frequency maps from Kelvin to Compton 
HI_857_map_enmap_40 = HI_857_map_enmap_40 * 0.0380

### Putting maps+mask into NaMaster

In [ ]:
#only with the least restrictive thresholded mask of 40e, just compare the relavent spectra
Y_object_1_full = nw.namap_car(maps = (ACT_Y_map_1, None, None), masks = HI_857_mask_enmap_40[0], beams = bl_ACT)
Y_object_2_cib = nw.namap_car(maps = (ACT_Y_map_2_cib, None, None), masks = HI_857_mask_enmap_40[0], beams = bl_ACT)

In [ ]:
cib_object_857_HI_map_40 = nw.namap_car(maps = (HI_857_map_enmap_40[0], None, None), masks = HI_857_mask_enmap_40[0], beams = beam_HI_857_40)

In [ ]:
mc_full_40 = nw.mode_coupling(Y_object_1_full, cib_object_857_HI_map_40, bins = bins, mcm_dir = path_beam + 'mc_full_40', overwrite = True)
mc_cib_40 = nw.mode_coupling(Y_object_2_cib, cib_object_857_HI_map_40, bins = bins, mcm_dir = path_beam + 'mc_cib_40', overwrite = True)

mc_HI_auto_40 = nw.mode_coupling(cib_object_857_HI_map_40, cib_object_857_HI_map_40, bins = bins, mcm_dir = path_beam + 'mc_HI_auto_40', overwrite = True)

mc_full_auto = nw.mode_coupling(Y_object_1_full, Y_object_1_full, bins = bins, mcm_dir = path_beam + 'mc_full_auto', overwrite = True)
mc_cib_auto = nw.mode_coupling(Y_object_2_cib, Y_object_2_cib, bins = bins, mcm_dir = path_beam + 'mc_cib_auto', overwrite = True)

In [ ]:
#cross spectra between the ACT Y maps and the HI_cleaned 857 GHz frequency map
CB_857_full_40 = nw.compute_spectra(Y_object_1_full, cib_object_857_HI_map_40, mc=mc_full_40, lmax = 3000, bins = bins)
CB_857_cib_40 = nw.compute_spectra(Y_object_2_cib, cib_object_857_HI_map_40, mc=mc_cib_40, lmax = 3000, bins = bins)

In [ ]:
#autospectra for the purpose of calculating variance and error 
CB_857_auto_40 = nw.compute_spectra(cib_object_857_HI_map_40, cib_object_857_HI_map_40, mc=mc_HI_auto_40, lmax = 3000, bins = bins)

In [ ]:
#for the purpose of errors
CB_full_auto = nw.compute_spectra(Y_object_1_full, Y_object_1_full, mc=mc_full_auto, lmax = 3000, bins = bins)
CB_cib_auto = nw.compute_spectra(Y_object_2_cib, Y_object_2_cib, mc=mc_cib_auto, lmax = 3000, bins = bins)

In [ ]:
f_sky_40 = 0.33826835950215656

bins_array = [15,15,20,20,20,25,25,25,25,75,75,100,150,150,200,200,200,200,200,200,200,200,200,200,200]

var_857_full_40 = (CB_857_full_40['TT']*CB_857_full_40['TT'] + CB_full_auto['TT']*CB_857_auto_40['TT'])/((2*CB_857_full_40['ell']+1)*f_sky_40*bins_array)
var_857_cib_40 = (CB_857_cib_40['TT']*CB_857_cib_40['TT'] + CB_cib_auto['TT']*CB_857_auto_40['TT'])/((2*CB_857_cib_40['ell']+1)*f_sky_40*bins_array)